<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-06-04 13:00:51
-------------------
qualified stocks: 93
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.03 L
Current:  1.32 C
-------------------
Today PnL: 32.84 K (0.25%)
Current PnL: -16.30 L (-11.36%)
CY Booked + Current PnL: -11.83 L (-8.24%)
-------------------
Total profit:  4.38 L
Total loss:  -20.69 L
-------------------
Total Booked + Current PnL: 15.15 L (13.0%)
Total Booked PnL: 31.45 L (26.99%)
Curr Year Booked PnL: 4.47 L (3.39%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.11 C
Est FTT PnL: 79.43 L (60.27%)
-------------------
Est LTT:  2.73 C
Est LTT PnL: 1.42 C (107.47%)
Deployed:  1.17 C
Current:  1.32 C
CAGR/XIRR %: 8.77%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PIDILITIND,2504.06,3202.00,-16.23,X-LC,38.71,92820.0,17698.0,3239.0,-0.07,23.56,3.49,27.87,9.0,5.46,0.71,16.41,X40,NTT,CHEMICALS
49,LICI,785.05,983.00,-3.65,H-LC,88.17,236716.0,42024.0,7078.0,0.09,21.58,2.99,25.21,31.0,5.94,1.81,30.90,XY25,NTT,INSURANCE
28,GILLETTE,8109.44,10330.69,13.01,H-SC,93.55,166744.0,28884.0,8871.0,1.67,20.95,5.32,27.39,116.0,3.26,1.28,39.51,X40,ATH,FMCG
4,ADANIPORTS,1103.69,1583.00,1.78,M-LC,59.14,101935.0,23573.0,10459.0,0.24,30.08,10.26,43.43,70.0,2.25,0.78,36.34,XY24,NTT,MISC
85,UNIONBANK,123.87,163.00,-0.31,M-LC,90.32,174643.0,33803.0,10688.0,1.29,24.00,6.12,31.59,79.0,3.16,1.34,51.70,XY24,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TITAGARH,1097.23,1548.00,-7.32,H-SC,96.77,162750.0,-29265.0,108147.0,3.48,-15.24,66.45,41.08,128.0,-0.27,1.24,37.75,XY24,NTT,ENGINEERING
59,RAJESHEXPO,517.65,583.00,1884.85,M-SC,60.22,54142.0,-83035.0,100352.0,3.47,-60.53,185.35,12.62,209.0,-0.83,0.41,34.65,X40N,NTT,JEWELLERY
29,GREENPANEL,375.16,537.00,186.18,M-SC,74.19,127807.0,-55271.0,134248.0,2.24,-30.19,105.04,43.14,191.0,-0.41,0.98,20.22,XY24,NTT,MISC
86,VAIBHAVGBL,350.21,521.00,65.84,X-SC,81.72,110648.0,-41343.0,115461.0,2.14,-27.20,104.35,48.77,60.0,-0.36,0.85,27.09,XR,NTT,JEWELLERY
15,BSOFT,628.85,844.81,-24.48,M-SC,36.56,70849.0,-39200.0,76992.0,1.88,-35.62,108.67,34.34,184.0,-0.51,0.54,16.64,XR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SATIN,199.77,274.0,-22.47,H-SC,6.45,152546.0,-43428.0,116255.0,-2.40,-22.16,76.21,37.16,122.0,-0.37,1.17,13.92,XY24,NTT,FINANCE
52,MASFIN,326.60,399.5,-24.12,H-SC,62.37,86100.0,-11880.0,33751.0,-1.70,-12.12,39.20,22.32,127.0,-0.35,0.66,24.87,XR,ATH,FINANCE
8,ATULAUTO,624.44,844.0,3505.41,M-SC,27.96,112336.0,-35032.0,86847.0,-1.31,-23.77,77.31,35.16,211.0,-0.40,0.86,14.74,XY24,NTT,AUTO
87,VALIANTORG,512.64,838.0,-377.60,H-SC,100.00,118092.0,-15194.0,99788.0,-1.15,-11.40,84.50,63.47,130.0,-0.15,0.90,97.95,XR,NTT,CHEMICALS
53,MEDANTA,1022.60,1486.0,3.99,X-MC,25.81,183145.0,27710.0,42728.0,-1.00,17.83,23.33,45.32,66.0,0.65,1.40,24.61,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1461.46,2275.00,-13.14,X-LC,50.54,278148.0,16547.0,129088.0,0.71,6.33,46.41,55.67,1.0,0.13,2.13,14.35,X40,BTT,IT
80,TCS,3794.03,4998.00,-19.05,X-LC,19.35,264092.0,-31842.0,125761.0,0.31,-10.76,47.62,31.73,2.0,-0.25,2.02,5.71,X40,BTT,IT
50,LTIM,5564.16,7197.33,-3.95,X-LC,77.42,185112.0,-15198.0,73989.0,1.60,-7.59,39.97,29.35,3.0,-0.21,1.42,27.88,X200,ATH,IT
6,ASIANPAINT,2961.56,4250.00,-23.13,X-LC,4.30,152769.0,-48617.0,136224.0,-0.40,-24.14,89.17,43.51,5.0,-0.36,1.17,5.37,X40,BTT,PAINTS
34,HINDUNILVR,2413.81,2723.00,-16.14,X-LC,26.88,263323.0,-7024.0,41658.0,-0.05,-2.60,15.82,12.81,6.0,-0.17,2.01,8.98,XY25,NTT,FMCG


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,14.89,H-SC,98.92,132802.0,35156.0,12523.0,1.41,36.00,9.43,48.82,134.0,2.81,1.02,47.53,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,13.01,H-SC,93.55,166744.0,28884.0,8871.0,1.67,20.95,5.32,27.39,116.0,3.26,1.28,39.51,X40,ATH,FMCG
85,UNIONBANK,123.87,163.00,-0.31,M-LC,90.32,174643.0,33803.0,10688.0,1.29,24.00,6.12,31.59,79.0,3.16,1.34,51.70,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.00,1.78,M-LC,59.14,101935.0,23573.0,10459.0,0.24,30.08,10.26,43.43,70.0,2.25,0.78,36.34,XY24,NTT,MISC
49,LICI,785.05,983.00,-3.65,H-LC,88.17,236716.0,42024.0,7078.0,0.09,21.58,2.99,25.21,31.0,5.94,1.81,30.90,XY25,NTT,INSURANCE


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,99.34,M-MC,83.87,226391.0,1429.0,166216.0,0.97,0.64,73.42,74.53,170.0,0.01,1.73,31.44,XY24,BTT,STEEL
51,LUXIND,1448.19,2442.00,40.00,M-SC,73.12,145084.0,7506.0,86905.0,1.64,5.46,59.90,68.62,183.0,0.09,1.11,25.78,X40N,NTT,TEXTILES
55,PFIZER,4245.50,6318.30,14.89,H-SC,98.92,132802.0,35156.0,12523.0,1.41,36.00,9.43,48.82,134.0,2.81,1.02,47.53,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,13.01,H-SC,93.55,166744.0,28884.0,8871.0,1.67,20.95,5.32,27.39,116.0,3.26,1.28,39.51,X40,ATH,FMCG
57,POLYCAB,5005.95,7541.35,5.29,H-MC,32.26,100062.0,14961.0,28137.0,0.26,17.58,28.12,50.65,90.0,0.53,0.77,25.78,X40N,ATH,CABLES


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,3.78,H-SC,66.67,116567.0,14563.0,30459.0,-0.75,14.28,26.13,44.14,112.0,0.48,0.89,32.84,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-27.01,M-MC,61.29,142148.0,12996.0,74073.0,0.26,10.06,52.11,67.42,156.0,0.18,1.09,38.25,XR,NTT,BANKS
38,INDIAMART,2327.09,4911.36,-51.65,H-SC,58.06,126129.0,2793.0,134176.0,0.77,2.26,106.38,111.05,108.0,0.02,0.96,23.58,AR,ATH,MISC
92,WIPRO,243.46,420.00,-10.62,M-LC,34.41,153369.0,2424.0,107036.0,0.47,1.61,69.79,72.51,68.0,0.02,1.17,15.16,XR,NTT,IT
1,AARTIIND,487.04,919.00,38.34,M-SC,54.84,79509.0,-1827.0,73967.0,1.17,-2.25,93.03,88.69,205.0,-0.02,0.61,34.07,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MASFIN,326.60,399.5,-24.12,H-SC,62.37,86100.0,-11880.0,33751.0,-1.70,-12.12,39.20,22.32,127.0,-0.35,0.66,24.87,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,1.78,M-LC,59.14,101935.0,23573.0,10459.0,0.24,30.08,10.26,43.43,70.0,2.25,0.78,36.34,XY24,NTT,MISC
84,UJJIVANSFB,52.77,60.0,35.36,M-SC,51.61,117288.0,-25191.0,44710.0,-0.16,-17.68,38.12,13.70,221.0,-0.56,0.90,38.43,X40N,NTT,BANKS
87,VALIANTORG,512.64,838.0,-377.60,H-SC,100.00,118092.0,-15194.0,99788.0,-1.15,-11.40,84.50,63.47,130.0,-0.15,0.90,97.95,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.3,14.89,H-SC,98.92,132802.0,35156.0,12523.0,1.41,36.00,9.43,48.82,134.0,2.81,1.02,47.53,X40,ATH,PHARMA


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.0,-377.60,H-SC,100.00,118092.0,-15194.0,99788.0,-1.15,-11.40,84.50,63.47,130.0,-0.15,0.90,97.95,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.3,14.89,H-SC,98.92,132802.0,35156.0,12523.0,1.41,36.00,9.43,48.82,134.0,2.81,1.02,47.53,X40,ATH,PHARMA
7,ASIANTILES,75.41,137.0,6500.00,M-SC,97.85,72438.0,-21372.0,97987.0,0.45,-22.78,135.27,81.67,214.0,-0.22,0.55,40.28,XR,NTT,CERAMICS
81,TITAGARH,1097.23,1548.0,-7.32,H-SC,96.77,162750.0,-29265.0,108147.0,3.48,-15.24,66.45,41.08,128.0,-0.27,1.24,37.75,XY24,NTT,ENGINEERING
74,STARHEALTH,551.56,761.0,4.66,M-SC,95.70,170955.0,-29261.0,105291.0,-0.34,-14.61,61.59,37.97,195.0,-0.28,1.31,37.26,XY24,NTT,INSURANCE


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.61
1,25,41.94
2,50,70.52


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    31.51
XY25    14.55
X40     14.45
AR      13.39
X40N    13.26
XR      10.35
X200     1.42
SR       1.12
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.22
X-LC    18.68
M-SC    17.39
H-MC     8.49
M-LC     7.69
H-LC     7.56
X-MC     5.54
M-MC     5.51
X-SC     2.76
L-SC     2.27
L-LC     1.11
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.70,-0.40,31.57
IT,10.17,-17.48,81.29
BANKS,9.70,-10.68,61.56
MISC,6.39,-29.19,86.33
FINANCE,5.82,-25.84,85.60
INSURANCE,5.29,3.67,25.21
PAINTS,5.26,-19.72,55.15
ELECTRICAL,5.09,-5.52,51.29
AUTO,4.30,-11.40,60.07


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2855950.0,26
AR,1231699.0,12
X40N,1219319.0,17
XR,1097447.0,14
X40,791422.0,11
XY25,475129.0,10
SR,198552.0,2
X200,73989.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2244323.0,21
M-SC,2044711.0,21
X-LC,866779.0,12
M-MC,548001.0,5
H-MC,516983.0,8
X-SC,348803.0,4
X-MC,308656.0,5
M-LC,301569.0,6
L-SC,300650.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      936375.0      6
M-SC       XY24      754090.0      6
H-SC       AR        544736.0      3
           X40N      501772.0      6
X-LC       X40       482806.0      5
M-SC       X40N      472988.0      6
M-MC       XY24      391745.0      3
M-SC       XR        291110.0      4
H-SC       XR        240046.0      4
M-SC       X40       225092.0      2
H-LC       AR        211864.0      3
M-LC       XY24      179751.0      4
H-MC       XY24      173560.0      2
X-LC       AR        173428.0      2
L-SC       XR        157161.0      2
X-MC       XY24      138345.0      2
X-LC       XY25      136556.0      4
M-SC       XY25      121648.0      1
X-SC       X40N      120439.0      2
           XR        115461.0      1
           SR        112903.0      1
H-MC       XR        112560.0      1
M-LC       XR        107036.0      1
X-MC       X40N       95983.0      2
M-SC       AR         94134.0      1
           SR         85649.0      1
M-MC       AR         82183.0      1
L-SC       XY24       81713.0      1
H-MC       XY25       77018.0      1
X-MC       XY25       74328.0      1
M-MC       XR         74073.0      1
X-LC       X200       73989.0      1
H-MC       AR         63578.0      1
           X40        62130.0      2
L-SC       AR         61776.0      1
L-LC       XY25       43719.0      1
H-MC       X40N       28137.0      1
H-SC       X40        21394.0      2
M-LC       XY25       14782.0      1
H-LC       XY25        7078.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
